# Neural Network
- *FFNN* : feed forward neural network (only execute forward propagation)
- Chapter5 -p301 explanation of theory (difficult...)

In [1]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

root = "./data"

transform = transforms.Compose(
    [transforms.ToTensor(), #Tensorオブジェクトに変換
    transforms.Normalize((0.5), (0.5)), #平均0.5, 標準偏差0.5で正規化
    lambda x: x.view(-1),] #データの形状を(28,28)から(784,)に変換
)

f_mnist_train = datasets.FashionMNIST(
    root=root,
    download=True,
    train=True,
    transform=transform,
)

f_mnist_test = datasets.FashionMNIST(
    root=root,
    download=True,
    train=False,
    transform=transform,
)
batch_size = 64

train_dataloader = DataLoader(
    f_mnist_train,
    batch_size=batch_size,
    shuffle=True,
)

test_dataloader = DataLoader(
    f_mnist_test,
    batch_size=batch_size,
    shuffle=False,
)

for (x,t) in train_dataloader:
    print(x.shape)
    print(t.shape)
    break

for (x,t) in test_dataloader:
    print(x.shape)
    print(t.shape)
    break

torch.Size([64, 784])
torch.Size([64])
torch.Size([64, 784])
torch.Size([64])


In [2]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.d1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.d1(x)
        x = self.fc2(x)
        return x


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(784, 256, 10).to(device)
model

MLP(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (d1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [4]:
import torch.optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [5]:
def train_step(x, t):
    model.train()
    preds = model(x)
    loss = criterion(preds, t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

In [6]:
def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

In [7]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=0):
        """
        patience: number of epochs observated
        verbose: frag of early stop message
        """
        self.epoch = 0
        self.pre_loss = float("inf")
        self.patience = patience
        self.verbose = verbose
    
    def __call__(self, current_loss):
        """
        current_loss: loss of test data after one epoch ran
        Return:
            true: the case that loss exceed the previous one by the limit of observation counts
            false:  
        """
        if self.pre_loss < current_loss:
            self.epoch += 1
        
        if self.epoch > self.patience:
            if self.verbose:
                print("early stopping")
            return True
        
        else:
            self.epoch = 0
            self.pre_loss = current_loss
        
        return False

In [8]:
%%time
from sklearn.metrics import accuracy_score

epochs = 200
history = {"loss":[], "accuracy":[], "test_loss":[], "test_accuracy":[]}
ers = EarlyStopping(patience=4, verbose=1)

for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    test_loss = 0.
    test_acc = 0.

    for (x,t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        test_loss += loss.item()
        train_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist(),
        )
    
    for (x,t) in test_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = test_step(x, t)
        test_loss += loss.item()
        
        test_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist(),
        )
    
    avg_train_loss = train_loss / len(train_dataloader)
    avg_train_acc = train_acc / len(train_dataloader)
    avg_test_loss = test_loss / len(test_dataloader)
    avg_test_acc = test_acc / len(test_dataloader)

    history["loss"].append(avg_train_loss)
    history["accuracy"].append(avg_train_acc)
    history["test_loss"].append(avg_test_loss)
    history["test_accuracy"].append(avg_test_acc)

    if (epoch + 1)%10 == 0:
        print(
            "epoch({}) train_loss:{:.4} train_acc: {:.4} val_loss: {:.4} val_acc: {:.4}".format(
                epoch+1,
                avg_train_loss,
                avg_train_acc,
                avg_test_loss,
                avg_test_acc
            )
        )
    if ers(avg_test_loss):
        break



KeyboardInterrupt: 